# Imports

In [1]:
# import some common libraries
import numpy as np
import os, cv2
import torch, torchvision
from zipfile import ZipFile
from PIL import Image
import os, sys

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg



# Model setup


In [2]:
# Set config 

cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2  # set threshold for this model
# Find a model from detectron2's model zoo
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
# Create Predictor

predictor = DefaultPredictor(cfg)

The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


# Loading our data


In [3]:
# Reading the zip file 

resp = "val2017.zip"

archive = ZipFile((resp))

count = 0  # Setting a number to limit the number of images we are going to read

# Creating a directory that contains images with sky removed

if not os.path.isdir( 'sky_treminated_images' ) :
    os.mkdir( 'sky_treminated_images' )  # make sure the directory exists


# Starting iteration on each image, each time an image with a sky is selected, we will DESTROY the sky in it

In [ ]:
# Iterating through the images, loading the using the existing model to make a prediction
# And than showing the image

for line in archive.namelist():
    if line != 'val2017/':

        #We will open the image inside the zip file, load it
        image_in_zip = archive.open(line)
        
        # We will use CV2 to decode the image
        im = cv2.imread(line)

        # Now we will use the model's predictor (pre-trained)
        panoptic_seg, segments_info = predictor(im)["panoptic_seg"]
        
        # We'll set the segments_info containing a dict to list, to make it easier to iterate it
        seg_list = list(segments_info)
        
        # i and j will be the sizes of our image shape        
        i = im.shape[1]
        j = im.shape[0]

        # now we will iterate on our seg_list, until we will meet a sky
        # Detectron2 will label the sky inside the dictinaries inside out seg_list
        # category_id = 40 is a sky, also, we want to know if the object IS NOT a thing (isthing = False)
        # isthing says the object is NOT a background if it sets to True
        sky_destroyed = False
        labels_tensor_to_numpy = panoptic_seg.cpu().numpy()
        for indx in range(len(seg_list)):
            sky_index = None
            if seg_list[indx]['isthing'] == False and seg_list[indx]['category_id'] == 40:
                sky_index = indx
            # If sky_index is None, we will not initiate the loop to terminate sky
            if sky_index != None:
                sky_destroyed = True
                for j1 in range(j):
                    for i1 in range(i):
                        if(labels_tensor_to_numpy[j1][i1] == sky_index+1):
                            #We will set the colour to 255, that is white (255,255,255)
                            im[j1,i1] = 255                         

        if sky_destroyed == True:
            
            cv2.imwrite(f'./sky_treminated_images/{line[8:-4]}.png',im)



c:\users\oriel\detectron2\detectron2\modeling\roi_heads\fast_rcnn.py:154: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()
